In [1]:
from dhanhq import dhanhq
import yaml
import pandas  as pd
import numpy as np
import time
import math




## Authentication

In [397]:
CONFIG_FILE_PATH = r"..\\configs\\config.yaml" #This has the API key and client id
with open(CONFIG_FILE_PATH,'r') as config:
    api_config = yaml.safe_load(config)
APPLICAITON_ID = api_config.get('api_config')[0]
CLIENT_ID = api_config.get('api_config')[1]
ACCESS_TOKEN = api_config.get('api_config')[2]
EXP_HOURS = api_config.get('api_config')[3]
dhan = dhanhq(CLIENT_ID,ACCESS_TOKEN)

## Getting other variables from configuration

In [365]:
OTHER_CONFIG_FILE_PATH = r"..\\configs\\other_config.yaml" # This we will configure all other variables
with open(OTHER_CONFIG_FILE_PATH,'r') as other_configs:
    gen_config = yaml.safe_load(other_configs)

In [366]:
underlying = gen_config.get('index_file_metadata')
options = gen_config.get('options_file_metadata')
trade_segment = gen_config.get('trade_segment')


In [367]:
underlying_file = pd.read_excel('underlying.xlsx')
options_file = pd.read_excel('Options.xlsx')


In [379]:
### Constant we are setting for that Options
UNDERLYING_INDEX = trade_segment['CRUDEOIL']['Name']
strike_size = trade_segment[UNDERLYING_INDEX]['StrikeSize']
index_segment = trade_segment[UNDERLYING_INDEX]['INDEX_SEG']
options_segment = trade_segment[UNDERLYING_INDEX]['OPTIONS_SEGMENT']
CAPITAL = 100000

### GET ITM ID

In [369]:
def get_index_ltp(dhanO, INDEX):
    INDEX_ID = str(underlying_file.loc[(underlying_file.SEM_TRADING_SYMBOL == INDEX),['SEM_SMST_SECURITY_ID']].squeeze())
    INDEX_LTP = dhanO.ohlc_data(securities = {f"{index_segment}":[int(INDEX_ID)]})['data']['data'][f'{index_segment}'][f'{INDEX_ID}']['last_price']
    return INDEX_LTP


In [370]:
def get_itm_strikes(spot,side,depth):
    i = -1*depth if side=="PE" else depth
    strike = int((spot- spot%strike_size)-(i*strike_size))
    return(strike)
    

In [371]:
def get_strike_id_lot_size(underlying,strike,side,series):
    return(options_file.loc[(options_file['SEM_UNDERLYING'] ==underlying) & (options_file['SEM_STRIKE_PRICE']==strike) & (options_file['SEM_OPTION_TYPE']==side) & (options_file['Series']==series),['SEM_SMST_SECURITY_ID','SEM_LOT_UNITS']].to_dict(orient='records')[0])

In [372]:
def get_options_ltp(dhanO,segment,strike_id, strike,capital,profit_multiple=1, stoploss_multiple=1):
    time.sleep(1)
    ohlc = dhanO.ticker_data(securities={f'{segment}':[strike_id]})
    ltp = ohlc['data']['data'][segment][f'{strike_id}']['last_price']
    profit=ltp*(1+profit_multiple*(1/100))
    sl=ltp*(1-stoploss_multiple*(1/100))
    qty = int(capital/ltp)
    return({'status':'success',
            'segment':segment,
             'strike':strike,
             'strike_id':strike_id,
             'qty':qty,
             'ltp':ltp,
             'profit':profit,
             'sl':sl
    })

In [384]:
219.60*(1-(1*(1/100)))

217.404

In [392]:
# placing order for MCX is differet than NIFTY Options
# MCX 
def place_limit_order_MCX(UNDERLYINGIDX,side,series,depth,trans="BUY", cap=CAPITAL, test=True):
    trans_type=dhanhq.BUY if trans == "BUY" else dhanhq.SELL
    exg_sgmt = dhanhq.MCX
    spot = get_index_ltp(dhan,UNDERLYING_INDEX ) # this will get the spot price
    strike = get_itm_strikes(spot,side,depth)  # this will get the strikr price
    strike_id_lotsize = get_strike_id_lot_size(UNDERLYING_INDEX,strike,side,series) # this will get Dhan Script ID for that strike and also LOT Size: We will be using it for placing order.
    strike_id = strike_id_lotsize['SEM_SMST_SECURITY_ID']
    lot_size = strike_id_lotsize['SEM_LOT_UNITS']
    price = get_options_ltp(dhan,options_segment,strike_id,strike,cap,1,1) # We will get the price, quantity and BO prices
    # Quantity shoul dbe divided by 100 and round it to integer.
    qty = int(price['qty']/100)
    entry_price = np.round(price['ltp'],1)
    sl = np.round(price['sl'],1)
    profit = np.round(price['profit'],1)
    print(entry_price)
    print(profit)
    sl_limit = sl - 0.3
    print(sl_limit)
    print(sl)

    # Placing the Buy Order
    buy_order_results = dhan.place_order(
         transaction_type=trans_type,
         exchange_segment=exg_sgmt,
         product_type=dhanhq.INTRA,
         order_type=dhanhq.LIMIT,
         validity=dhanhq.DAY,
         security_id=strike_id,
         quantity=1 if test else qty,
         price=entry_price,
         #bo_profit_value=price['profit'],
         #bo_stop_loss_Value=price['sl']
         )
    time.sleep(1)

    #Placing the Exit Orders
    if buy_order_results['status']=='success':
       sell_order_sl = dhan.place_order(
         transaction_type=dhanhq.SELL,
         exchange_segment=exg_sgmt,
         product_type=dhanhq.INTRA,
         order_type=dhanhq.SL,
         validity=dhanhq.DAY,
         security_id=strike_id,
         quantity=1 if test else qty,
         price=sl_limit,
         trigger_price =sl,
         tag="sl"

         )
       sell_order_profit = dhan.place_order(
         transaction_type=dhanhq.SELL,
         exchange_segment=exg_sgmt,
         product_type=dhanhq.INTRA,
         order_type=dhanhq.LIMIT,
         validity=dhanhq.DAY,
         security_id=strike_id,
         quantity=1 if test else qty,
         price=profit,
         tag="profit"
         )
    overall_order_status = {'entry':buy_order_results,'sl':sell_order_sl,'profit':sell_order_profit}
    
    # Order Management
    entry_order_id = overall_order_status['entry']['data']['orderId']
    sl_order_id = overall_order_status['sl']['data']['orderId']
    profit_order_id = overall_order_status['profit']['data']['orderId']
    if (overall_order_status['entry']['status']=='success' and overall_order_status['sl']['status']=='success' and overall_order_status['profit']['status']=='success'):
      k = True
      while True and k:
         time.sleep(3)
         sl_order_details = dhan.get_order_by_id(sl_order_id)['data'][0]['orderStatus']
         profit_order_details = dhan.get_order_by_id(profit_order_id)['data'][0]['orderStatus']
         if sl_order_details == 'TRADED':
            canelled_profit_order = dhan.cancel_order(profit_order_id)
            canelled_sl_order =''
            capital_protection_exit = ''
            k=False
         if profit_order_details == 'TRADED':
            canelled_sl_order = dhan.cancel_order(sl_order_id)
            canelled_profit_order = ''
            capital_protection_exit = ''
            k=False
    else:
      sl_order_details = dhan.get_order_by_id(sl_order_id)['data'][0]['orderStatus']
      profit_order_details = dhan.get_order_by_id(profit_order_id)['data'][0]['orderStatus']
      entry_order_details = dhan.get_order_by_id(entry_order_id)['data'][0]['orderStatus']
      if (sl_order_details == 'REJECTED' or sl_order_details == 'CANCELLED') :
        canelled_profit_order = dhan.cancel_order(profit_order_id)
        canelled_sl_order = ''
        capital_protection_exit = ''
      if (profit_order_details == 'REJECTED'  or profit_order_details == 'CANCELLED'):
        canelled_sl_order = dhan.cancel_order(sl_order_id)
        canelled_profit_order = ''
        capital_protection_exit = ''
      if entry_order_details == 'TRADED':
        capital_protection_exit =  dhan.place_order(
          transaction_type=dhanhq.SELL,
          exchange_segment=exg_sgmt,
          product_type=dhanhq.INTRA,
          order_type=dhanhq.MARKET,
          validity=dhanhq.DAY,
          security_id=strike_id,
          quantity=1 if test else qty,
          price=0,
          )
      
    
    #Outut
    order_exits = {
       'sl_canceled':canelled_sl_order,
       'profit_canceled':canelled_profit_order,
       'capital_protection_exit':capital_protection_exit

       
    }
    
    print("Order Entry", overall_order_status)
    print("Order Management", order_exits)
#     if(overall_order_status['entry']])
         

    return {'entries':overall_order_status,'order_management':order_exits}



In [ ]:
def place_bo_order_FNO(UNDERLYINGIDX,side,series,depth,trans="BUY", cap=CAPITAL, test=True):
    trans_type=dhanhq.BUY if trans == "BUY" else dhanhq.SELL
    exg_sgmt = dhanhq.NSE_FNO
    spot = get_index_ltp(dhan,UNDERLYING_INDEX ) # this will get the spot price
    strike = get_itm_strikes(spot,side,depth)  # this will get the strikr price
    strike_id_lotsize = get_strike_id_lot_size(UNDERLYING_INDEX,strike,side,series) # this will get Dhan Script ID for that strike and also LOT Size: We will be using it for placing order.
    strike_id = strike_id_lotsize['SEM_SMST_SECURITY_ID']
    print(strike_id)
    lot_size = strike_id_lotsize['SEM_LOT_UNITS']
    price = get_options_ltp(dhan,options_segment,strike_id,strike,cap,1,1) # We will get the price, quantity and BO prices
    # Quantity shoul dbe divided by 100 and round it to integer.
    qty = int(cap/price['ltp']/lot_size)*lot_size
    entry_price = np.round(price['ltp'],1)-0.05
    sl = entry_price - np.round(price['sl'],1)-0.05
    profit = np.round(price['profit'],1)+0.05 - entry_price

    # Placing the Buy Order

    buy_order_results = dhan.place_slice_order(
         transaction_type=trans_type,
         exchange_segment=exg_sgmt,
         product_type=dhanhq.BO,
         order_type=dhanhq.LIMIT,
         validity=dhanhq.DAY,
         security_id=strike_id,
         quantity=lot_size if test else qty,
         price=entry_price,
         bo_profit_value=profit,
         bo_stop_loss_Value=sl
         )

    return(buy_order_results)

In [387]:
196*(1-(1*(1/100)))

194.04

In [393]:
orders= place_limit_order_MCX(UNDERLYING_INDEX,'PE','C',1,"BUY")


194.4
196.3
192.2
192.5
Order Entry {'entry': {'status': 'success', 'remarks': '', 'data': {'orderId': '48112211245213', 'orderStatus': 'TRANSIT'}}, 'sl': {'status': 'success', 'remarks': '', 'data': {'orderId': '18112211248613', 'orderStatus': 'TRANSIT'}}, 'profit': {'status': 'success', 'remarks': '', 'data': {'orderId': '58112211247113', 'orderStatus': 'TRANSIT'}}}
Order Management {'sl_canceled': '', 'profit_canceled': {'status': 'success', 'remarks': '', 'data': {'orderId': '58112211247113', 'orderStatus': 'CANCELLED'}}, 'capital_protection_exit': ''}


In [376]:
orders = place_bo_order_FNO(UNDERLYING_INDEX,'CE','C',1,"BUY")

42935


In [394]:
orders

{'entries': {'entry': {'status': 'success',
   'remarks': '',
   'data': {'orderId': '48112211245213', 'orderStatus': 'TRANSIT'}},
  'sl': {'status': 'success',
   'remarks': '',
   'data': {'orderId': '18112211248613', 'orderStatus': 'TRANSIT'}},
  'profit': {'status': 'success',
   'remarks': '',
   'data': {'orderId': '58112211247113', 'orderStatus': 'TRANSIT'}}},
 'order_management': {'sl_canceled': '',
  'profit_canceled': {'status': 'success',
   'remarks': '',
   'data': {'orderId': '58112211247113', 'orderStatus': 'CANCELLED'}},
  'capital_protection_exit': ''}}

In [340]:
int(100000/194/25)*25

500

In [361]:
dhan.ticker_data(securities={'NSE_FNO':[42935]})['data']['data']['NSE_FNO']['42935']['last_price']



221.3

In [391]:
dhanhq.__dict__

mappingproxy({'__module__': 'dhanhq.dhanhq',
              '__firstlineno__': 18,
              '__doc__': 'DhanHQ Class to interact with REST APIs',
              'NSE': 'NSE_EQ',
              'BSE': 'BSE_EQ',
              'CUR': 'NSE_CURRENCY',
              'MCX': 'MCX_COMM',
              'FNO': 'NSE_FNO',
              'NSE_FNO': 'NSE_FNO',
              'BSE_FNO': 'BSE_FNO',
              'INDEX': 'IDX_I',
              'BUY': 'BUY',
              'SELL': 'SELL',
              'CNC': 'CNC',
              'INTRA': 'INTRADAY',
              'MARGIN': 'MARGIN',
              'CO': 'CO',
              'BO': 'BO',
              'MTF': 'MTF',
              'LIMIT': 'LIMIT',
              'MARKET': 'MARKET',
              'SL': 'STOP_LOSS',
              'SLM': 'STOP_LOSS_MARKET',
              'DAY': 'DAY',
              'IOC': 'IOC',
              'COMPACT_CSV_URL': 'https://images.dhan.co/api-data/api-scrip-master.csv',
              'DETAILED_CSV_URL': 'https://images.dhan.co/api-